# **Neural Style Transfer with Lille images using Pytorch**
---






In [0]:
# import resources
%matplotlib inline

import os

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

from instaloader import Instaloader, Profile
from fastai.vision import *
from fastai.vision import verify_images
from pathlib import Path
import itertools

import torch
import torch.optim as optim
from torchvision import *

In [0]:
current_path = Path('/content/drive/My Drive/')

## **Download requirements**
---

In [29]:
!pip install -r "/content/drive/My Drive/resources/requirements.txt"

  Cloning https://github.com/bleshik/instaloader.git to /tmp/pip-req-build-p9tkei04
  Running command git clone -q https://github.com/bleshik/instaloader.git /tmp/pip-req-build-p9tkei04
  Created wheel for instaloader: filename=instaloader-4.2.6-cp36-none-any.whl size=41606 sha256=e862f4e322642cc7639389929c77af94c0e811728bb3910751a44a83f118d0dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-2jwjgiyx/wheels/6b/99/f9/f37a7414fbe8f3b65e10fff1997066aa80618578a9e0ba3687
Successfully built instaloader


##**Download images**

### **Download Lille images (content)**
---









In [0]:
# Get current directory
content_path = Path('drive/My Drive/data/content/')
folder = "dataset_lille_images"
txt_file_name = "lille_images_urls.txt"

# Create the directories to store the images
os.makedirs(path/folder, exist_ok=True)

Get URLs images of instagram posts using instaloader library and write them into a txt file

In [0]:
loader = Instaloader(download_videos=False,
                 download_video_thumbnails=False,
                 download_geotags=False,
                 download_comments=False,
                 save_metadata=False,
                 compress_json=False,)

profile_name = "lillemaville"
profile = Profile.from_username(loader.context, profile_name)

In [0]:
file = open(path/txt_file_name, 'w')

for post in itertools.islice(profile.get_posts(), 50):
    file.write("%s\n" % post.url)
    
file.close()

Download the retrieved images with Fastai

In [34]:
path_content_imgs = content_path/folder

print("Downloading Lille Images...")
download_images(urls=content_path/txt_file_name, dest=path_content_imgs)

# verify images have correct properties for training
print("Delete Corrupted Images...")
verify_images(path_content_imgs, delete=True, img_format=f'{folder} %d')

Delete Corrupted Images...


### **Download style images**

In [30]:
!wget https://oceanmhs.org/wp-content/uploads/2018/01/starrynight.jpg

--2020-02-06 14:01:39--  https://oceanmhs.org/wp-content/uploads/2018/01/starrynight.jpg
Resolving oceanmhs.org (oceanmhs.org)... 104.155.134.146
Connecting to oceanmhs.org (oceanmhs.org)|104.155.134.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92567 (90K) [image/jpeg]
Saving to: ‘starrynight.jpg’

starrynight.jpg     100%[===================>]  90.40K   352KB/s    in 0.3s    

2020-02-06 14:01:40 (352 KB/s) - ‘starrynight.jpg’ saved [92567/92567]



## **Data exploration** - Looking at the data...

In [0]:
#TODO : Data visualisation of Lille images

## **Data Loader**

In [0]:
# desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

VGG networks are trained on images with each channel normalized by mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225]. We will use them to normalize the image before sending it into the network.

In [0]:
transform = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])  # transform it into a torch tensor

In [0]:
content_data = datasets.ImageFolder("/content/drive/My Drive/data/content", transform=transform)

In [70]:
dataloader = DataLoader(content_data, batch_size=4, shuffle=True, num_workers=2)
print(f"Number of Lille images: {len(dataloader.dataset)}")

Number of Lille images: 50


In [85]:
# dataset_subset = torch.utils.data.RandomSampler(content_data, replacement=True, num_samples=1)
# dataset_subset.dataset

1

## **Importing the model** (test VGG19)

In [0]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        # Load VGG Skeleton (only convolutional and pooling layers)
        self.features = models.vgg19(pretrained=True).features

        # Freeze all VGG parameters
        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        layers = {'0': 'conv1_1',
                  '5': 'conv2_1',
                  '10': 'conv3_1',
                  '19': 'conv4_1',
                  '21': 'conv4_2', ## content representation
                  '28': 'conv5_1'}

        features = {}
        for name, layer in self.features._modules.items():
            x = layer(x)
            if name in layers:
                features[layers[name]] = x

        return features

In [0]:
# style = load_image("/content/drive/My Drive/data/style/starrynight.jpg")
# model = VGG19()
# style_features = model(style)

In [107]:
# style_features

{'conv1_1': tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.6605, 0.8130, 0.5558],
           [0.0000, 0.0000, 0.0000,  ..., 0.5165, 0.5896, 0.3959],
           [0.0418, 0.0000, 0.0000,  ..., 0.7128, 0.9100, 0.5979],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
          [[1.6026, 0.0000, 0.1762,  ..., 1.5498, 1.4392, 4.4417],
           [3.9469, 1.4979, 1.5243,  ..., 0.0000, 0.0000, 3.7629],
           [2.1865, 1.9848, 0.4359,  ..., 0.0000, 0.0000, 3.9241],
           ...,
           [4.0430, 0.5507, 0.6079,  ..., 0.3050, 0.3050, 0.0000],
           [4.0063, 0.5335, 0.6126,  ..., 0.3050, 0.3050, 0.0000],
           [5.0541, 3.2208, 3.3730,  ..., 3.3390, 3.3390, 0.1079]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 0.9293, 1.2137, 0.6682],
           [0.0000, 0.0000, 0.0000,  ..., 0.7231, 1.0291, 0.5766],
           [0

Gram Matrix

Loss and Weights

Optimizer (?)

Updating the Target & Calculating Losses

Save the target image